In [12]:
import matplotlib.pyplot as plt
import pickle
import os
import math
import numpy as np
from od.social import SocialGroup
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.misc.statistic import NetFlowType
from numpy import random
from od.social import SocialGroup
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.config import BS_PRESET,SUMO_SIM_SECONDS
import od.vars as GV
plt.rcParams.update(
    {
    "text.usetex": False,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "legend.loc":"upper left",
    "legend.framealpha":0.3,
    "font.size":14,
    }
)
os.getcwd()

'/home/OD/repos/ProjectSumo'

In [13]:
class Printer:
        def Log(self,data):
                pass
                # print(data)
        def Doc(self,data):
                pass
                # print(data)
class FakeBSController:
        def __init__(self,serial,bs_config):
                self.serial = serial 
                self.type=  bs_config["type"]
        def __index__(self):
                return self.serial
def InitGV(interest_config):
        GV.NET_STATION_CONTROLLER = [
                FakeBSController(serial,bs_config) for serial,bs_config in enumerate(BS_PRESET.values())
                if ((bs_config["type"] == BaseStationType.UMA)  or 
                (interest_config.req_rsu and bs_config["type"] == BaseStationType.UMI))
        ]
        GV.STATISTIC =  Printer()

In [14]:
seed_range = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
res_alloc_type_range = [ResourceAllocatorType.NOMA_OPT]
req_rsu_range = [True,False]
# traffic_scale_range = [0.4, 0.6,0.8,1.0,1.2,1.4]
traffic_scale_range=[1.6]
# seed_range = [4]
# res_alloc_type_range = [ResourceAllocatorType.OMA]
# req_rsu_range = [True]
# traffic_scale_range = [1.1]

In [15]:
def Process(app_stats):
        for header,record in app_stats.items():
            for bs in GV.NET_STATION_CONTROLLER:
                for pair in record.time_bs_txq[bs]:
                    pair[0] =  math.ceil(round(pair[0],4)*1000)/1000
                    pair[1] =  math.ceil(round(pair[1],4)*1000)/1000
            

In [16]:
interest_config = InterestConfig(None, False, 0)
for seed in seed_range:
    for res_alloc_type in res_alloc_type_range:
        for req_rsu in req_rsu_range:
            for traffic_scale in traffic_scale_range:
                interest_config.res_alloc_type = res_alloc_type
                interest_config.req_rsu = req_rsu
                interest_config.traffic_scale = traffic_scale
                try:
                    stats_report = None
                    stats_record = None
                    sum_wait_time = [None for nft in NetFlowType]
                    InitGV(interest_config)
                    with open("data/yQoS/{}/{}/report.pickle".format(seed,str(interest_config)), "rb") as file:
                        stats_report = pickle.load(file)
                    with open("data/yQoS/{}/{}/object.pickle".format(seed,str(interest_config)), "rb") as file:
                        stats_record = pickle.load(file)
                    
                    # Process(stats_record.ExtractNetworkTraffic(NetFlowType.GENERAL))
                    for nft in NetFlowType:
                        nft_flow = stats_record.ExtractNetworkTraffic(nft)
                        Process(nft_flow)
                        sum_wait_time[nft] = stats_record.BaseStationAppdataTXQReport(nft_flow)
                    rep_wait_time = {
                        x.name: sum_wait_time[x]
                        for x in NetFlowType
                    }
                    stats_report["wait-time"] =rep_wait_time
                    with open(
                        "data/yQoS/{}/{}/report_adjust_wait_time_nQoS.pickle".format(seed,str(interest_config)), "wb"
                        ) as file:

                        pickle.dump(stats_report,file)
                except FileNotFoundError:
                    print("missing: {} seed({})".format(str(interest_config),seed))